## Imports

In [1]:
import numpy as np
import random
import operator
import time
import pandas as pd
from wordle_functions import *

## Importing datasets

### official words
- official wordle word list

In [2]:
### Official list
official_words = []

with open("data/official_words_processed.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        official_words.append(word)

f.close() # closes connection to file

print(len(official_words))
official_words[:5]

2310


['foist', 'dowdy', 'bleat', 'basis', 'tango']

### alternative list 1
- an alternate list of 5-letter words found on the web

In [3]:
### Official list
alt_words_1 = []

with open("data/alt_words_1.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        alt_words_1.append(word)

f.close() # closes connection to file

print(len(alt_words_1))
alt_words_1[:5]

14856


['rossa', 'jetty', 'wizzo', 'cuppa', 'cohoe']

### nltk grand corpus
- Amalgamation of all words in various NLTK corpora to have as big a dataset as possible
- Developed manually

In [4]:
### grand corpus tokens
nltk_tokens = []

with open("data/nltk_grand_corpus_tokens.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        nltk_tokens.append(word)

f.close() # closes connection to file

print(len(nltk_tokens))
nltk_tokens[:5]

535189


['years', 'board', 'dutch', 'group', 'agnew']

### nltk grand corpus types and counts

In [5]:
### grand corpus types and counts
nltk_counts = {}

with open("data/nltk_grand_corpus_types_and_counts.txt", "r", encoding = "utf-8") as f:
    for line in f.read().split("\n"):
        if len(line.split("\t")) == 2:
            word = line.split("\t")[0]
            count = line.split("\t")[1]
            nltk_counts[word] = count
        else:
            continue

f.close() # closes connection to file

print(len(nltk_counts))
nltk_counts['which']

8043


'15760'

In [6]:
### Official list
official_words = []

with open("data/official_words_processed.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        if len(word) > 0: # there's one blank entry at the start
            official_words.append(word)

f.close() # closes connection to file

print(len(official_words))
official_words[:10]

2309


['foist',
 'dowdy',
 'bleat',
 'basis',
 'tango',
 'eking',
 'knead',
 'power',
 'dwell',
 'bleep']

## `user_wordle()` function
- still workshopping this. Goal is to compare a given user's guesses against how `wordle_wizard()` would play it

In [7]:
# def user_wordle(word_list: list, max_guesses: int = None, 
#                 guess: str = None, target: str = None, bias: bool = True, 
#                 random_guess: bool = False, random_target: bool = False, 
#                 verbose: bool = False, drama: float = None, 
#                 return_stats: bool = False, record: bool = False):
#     """
#     Unlike the `wordle_wizard()` function, that provides post-starting guesses automatically, this function can take each guess from the user and evaluate next options accordingly.

#     ------
#     Parameters:
#     ------
#     `word_list`: list
#         list of valid words to be considered
#     `guess`: str
#         a string -- must be the same length as `target_word`
#     `target`: str
#         a string -- must be the same length as `opening_word`
#     `bias`: str ['entropy', 'common', 'rare', None]
#         'entropy' biases next word guesses to be the ones with the highest impact on the range of next possible guesses.

#         'common' biases next word guesses to be words that are more commonly used

#         'rare' biases next word guesses to be words that are more rarely used

#         'no_bias' chooses a next guess at random of all available guesses
#     `max_guesses`: int
#         the maximum number of attempts allowed to solve the Wordle
#     `random_guess`: bool
#         if True, randomly chooses a starting word from all words within `word_list`. If False, passed starting word must be used instead
#     `random_target`: bool
#         if True, randomly chooses a target word from all words within `word_list`. If False, passed target word must be used instead
#     `verbose`: bool
#         if True, prints progress and explanation of how function solves the puzzle. If False, prints only the guessed word at each guess
#     `drama`: float or int
#         if int provided, each guess' output is delayed by that number of seconds, else each output is shown as quickly as possible. For ~dRaMaTiC eFfEcT~
#     `return_stats`: bool
#         if True, prints nothing and returns a dictionary of various statistics about the function's performance trying to solve the puzzle
#     `record`: bool
#         if True, creates a .txt file matching the printed output of the function, else does nothing

#     ------
#     Returns:
#     ------
#     `stats_master`: dict
#         dictionary containing various statistics about the user performance trying to solve the puzzle, alongside `wordle_wizard()` solving the same puzzle
#     """

#     stats_dict = {}

#     first_guess = []
#     if guess == "user_entered":
#         first_guess.append(guess)
#         guess = input("Please enter your starting guess").lower()
#     else:
#         guess = guess.lower()
#         first_guess.append(guess)

#     if random_guess == True:
#         randomint_guess = random.randint(0, len(word_list) - 1)
#         guess = word_list[randomint_guess]

#     if random_target == True:
#         randomint_target = random.randint(0, len(word_list) - 1)
#         target = word_list[randomint_target]
        
#     stats_dict = {}
#     stats_dict['first_guess'] = guess
#     stats_dict['target_word'] = target
#     stats_dict['first_guess_vowels'] = float(count_vows_cons(guess, y_vow = True)['vows'])
#     stats_dict['first_guess_consonants'] = float(count_vows_cons(guess, y_vow = True)['cons'])
#     stats_dict['target_vowels'] = float(count_vows_cons(target, y_vow = True)['vows'])
#     stats_dict['target_consonants'] = float(count_vows_cons(target, y_vow = True)['cons'])

#     # get entropy of the first guess word and target word in the entire word_list
#     for tup in get_word_entropy(word_list, word_list, normalized = True):
#         if tup[0] == guess:
#             stats_dict['first_guess_entropy'] = tup[1]
#         if tup[0] == target:
#             stats_dict['target_entropy'] = tup[1]
        
#     guess_entropies = []
#     guess_entropies.append(stats_dict['first_guess_entropy'])

#     english_alphabet = "abcdefghijklmnopqrstuvwxyz"
#     word_list_sorted_counts = get_letter_counts(english_alphabet, word_list, sort = "descending")
    
#     wordlen = len(guess)
#     letter_positions = set(i for i in range(0, wordlen))

#     guess_set = set()
#     perfect_dict = {}
#     wrong_pos_dict = {}
#     wrong_pos_set = set()
#     dont_guess_again = set()

#     guessed_words = [] # running set of guessed words
#     guess_num = 0 # baseline for variable
#     dont_guess_words = set()
#     incorrect_positions = []
#     reduction_per_guess = []

#     if max_guesses == None: # if no value is passed, default is len(guess)
#         max_guesses = wordlen
#     else: # else it is the value passed
#         max_guesses = max_guesses

#     perfect_letts_per_guess = []
#     wrong_pos_per_guess = []
#     wrong_letts_per_guess = []
#     record_list = []

#     while guess: # while there is any guess -- there are conditions to break it at the bottom

#         guess_num += 1

#         guessed_words.append(guess)

#         if drama:
#             time.sleep(drama)

#         # guess_num += 1 # each time the guess is processed
#         if return_stats == False:
#             if guess_num == 1:
#                 print("-----------------------------\n")
#                 record_list.append("-----------------------------\n")
    
#         if return_stats == False:
#             print(f"Guess {guess_num}, you entered: '{guess}'\n")
#             record_list.append(f"Guess {guess_num}, you entered: '{guess}'\n")

#         if guess == target:
#             stats_dict['target_guessed'] = True
#             if return_stats == False:
#                 if guess_num == 1:
#                     print(f"Congratulations! The Wordle has been solved in {guess_num} guess, that's amazingly lucky!")
#                     print(f"The target word was {target}")
                    
#                     record_list.append(f"Congratulations! The Wordle has been solved in {guess_num} guess, that's amazingly lucky!")
#                     record.append(f"The target word was {target}")
#                     perfect_letts_per_guess.append(5)
#                     wrong_pos_per_guess.append(0)
#                     wrong_letts_per_guess.append(0)
#             break

#         guess_set = set()
#         wrong_pos_set = set()

#         #### Step 2 -- ALL PERFECT
#         for i in letter_positions: # number of letters in each word (current word and target word)
#             guess_set.add(guess[i])

#             if guess[i] not in perfect_dict:
#                 perfect_dict[guess[i]] = set()
#             if guess[i] not in wrong_pos_dict:
#                 wrong_pos_dict[guess[i]] = set()

#             ### EVALUATE CURRENT GUESS
#             if guess[i] == target[i]: # letter == correct and position == correct
#                 perfect_dict[guess[i]].add(i)

#             if (guess[i] != target[i] and  guess[i] in target): # letter == correct and position != correct
#                 wrong_pos_dict[guess[i]].add(i)
#                 wrong_pos_set.add(guess[i])

#             if guess[i] not in target: # if letter is not relevant at all
#                 dont_guess_again.add(guess[i])

#         #### Step 3 -- ALL PERFECT
#         next_letters = set()
#         for letter, positions in perfect_dict.items():
#             if len(positions) > 0:
#                 next_letters.add(letter)

#         for letter, positions in wrong_pos_dict.items():
#             if len(positions) > 0:
#                 next_letters.add(letter)

#         #### List of tuples of correct letter positions in new valid words. Eg: [('e', 2), ('a', 3)]
#         perfect_letters = []
#         for letter, positions in perfect_dict.items():
#             for pos in positions:
#                 if len(positions) > 0:
#                     perfect_letters.append((letter, pos))

#         #### all words that have correct letters in same spots
#         words_matching_correct_all = []
#         for word in word_list:
#             word_set = set()
#             for letter, pos in perfect_letters:
#                 if word[pos] == letter:
#                     words_matching_correct_all.append(word)

#         #### excluding words with letters in known incorrect positions
#         for letter, positions in wrong_pos_dict.items():
#             for pos in positions:
#                 if len(positions) > 0:
#                     if (letter, pos) not in incorrect_positions:
#                         incorrect_positions.append((letter, pos))

#         # sorting lists of tuples just to make them look nice in the printout
#         incorrect_positions = sorted(incorrect_positions, key = operator.itemgetter(1), reverse = False)
#         perfect_letters = sorted(perfect_letters, key = operator.itemgetter(1), reverse = False)

#         #### all words that have correct letters in incorrect spots -- so they can be excluded efficiently
#         for word in word_list:
#             word_set = set()
#             for letter, pos in incorrect_positions:
#                 if word[pos] == letter:
#                     dont_guess_words.add(word)

#         for bad_letter in dont_guess_again:
#             for word in word_list:
#                 if (bad_letter in word and word not in dont_guess_words):
#                     dont_guess_words.add(word)

#         if return_stats == False:
#             if verbose == True:
#                 print(f"Letters in correct positions:\n\t{perfect_letters}\n")
#                 print(f"Letters in incorrect positions:\n\t{incorrect_positions}\n")
#                 print (f"Letters to guess again:\n\t{sorted(list(next_letters), reverse = False)}\n")
#                 print(f"Letters to not guess again:\n\t{sorted(list(dont_guess_again), reverse = False)}\n")

#                 record_list.append(f"Letters in correct positions:\n\t{perfect_letters}\n")
#                 record_list.append(f"Letters in incorrect positions:\n\t{incorrect_positions}\n")
#                 record_list.append(f"Letters to guess again:\n\t{sorted(list(next_letters), reverse = False)}\n")
#                 record_list.append(f"Letters to not guess again:\n\t{sorted(list(dont_guess_again), reverse = False)}\n")

#         # Returns True
#         # print(A.issubset(B)) # "if everything in A is in B", returns Bool

#         perfect_letts_per_guess.append(len(perfect_letters))
#         wrong_pos_per_guess.append(len(incorrect_positions))
#         wrong_letts_per_guess.append(len(dont_guess_again))

#         potential_next_guesses = set()
#         middle_set = set()

#         if len(perfect_letters) == 0 and len(incorrect_positions) == 0: # if there are NEITHER perfect letters, NOR incorrect positions, ....
#             for word in word_list:
#                 if word not in dont_guess_words:
#                     if word not in guessed_words:
#                         potential_next_guesses.add(word)
                                        
#             # print(f"GUESS {guess_num} : TEST 1-1")

#         if len(perfect_letters) == 0 and len(incorrect_positions) != 0: # if there are no perfect letters whatsoever, but there ARE incorrect positions ....
#             for word in word_list:
#                 for incor_letter, incor_pos in incorrect_positions:
#                     if word[incor_pos] != incor_letter:
#                         if word not in dont_guess_words: # just in case
#                             word_set = set()
#                             for letter in word:
#                                 word_set.add(letter)

#                                 if next_letters.issubset(word_set):
#                                     if word not in guessed_words:
#                                         if len(dont_guess_again) > 0:
#                                             for bad_letter in dont_guess_again:
#                                                 if bad_letter not in word:
#                                                     # potential_next_guesses.append(word)
#                                                     potential_next_guesses.add(word)
#                                         else:
#                                             potential_next_guesses.add(word)
            
#             # print(f"GUESS {guess_num} : TEST 2-1")

#         else:
#             for word in word_list:
#                 if word not in dont_guess_words: # just in case
#                     word_set = set()
#                     for letter in word:
#                         word_set.add(letter)
#                         if next_letters.issubset(word_set):
#                             if word not in guessed_words:
#                                 # print ("TEST 3-2")

#                                 if len(dont_guess_again) > 0:
#                                     for bad_letter in dont_guess_again:
#                                         if bad_letter not in word:
#                                             middle_set.add(word)
#                                 else:
#                                     middle_set.add(word)
#             for word in middle_set:
#                 dummy_list = []
#                 for good_lett, good_pos in perfect_letters:
#                     if word[good_pos] == good_lett:
#                         dummy_list.append(1)
#                         if len(dummy_list) == len(perfect_letters):
#                             potential_next_guesses.add(word)
#             for word in middle_set:
#                 dummy_list = []
#                 for bad_lett, bad_pos in incorrect_positions:
#                     if word[bad_pos] == bad_lett:
#                         dummy_list.append(1)
#                         if len(dummy_list) > 0:
#                             potential_next_guesses.remove(word)
                                        
#             # print(f"GUESS {guess_num} : TEST 3-1")

#         if return_stats == False:
#             if verbose == True:
#                 print(f"At this point:")
#                 print(f"\t{len(word_list) - len(potential_next_guesses)}, {round((len(word_list) - len(potential_next_guesses)) / len(word_list) * 100, 2)}% of total words have been eliminated, and")
#                 print(f"\t{len(potential_next_guesses)}, {round(len(potential_next_guesses) / len(word_list) * 100, 2)}% of total words remain possible.\n")

#                 record_list.append(f"At this point:")
#                 record_list.append(f"\t{len(word_list) - len(potential_next_guesses)}, {round((len(word_list) - len(potential_next_guesses)) / len(word_list) * 100, 2)}% of total words have been eliminated, and")
#                 record_list.append(f"\t{len(potential_next_guesses)}, {round(len(potential_next_guesses) / len(word_list) * 100, 2)}% of total words remain possible.\n")

#         reduction_per_guess.append(len(potential_next_guesses))

#         #### Guessing next word
#         if len(potential_next_guesses) == 1:

#             if return_stats == False:
#                 if verbose == True:
#                     print(f"The only remaining possible word is:\n\t'{list(potential_next_guesses)[0]}'\n")
#                     record_list.append(f"The only remaining possible word is:\n\t'{list(potential_next_guesses)[0]}'\n")
                
#             # guess = list(potential_next_guesses)[0]
#             if guess_num == max_guesses - 1:
#                 guess = input("This is your last guess, choose wisely").lower()
#             else:
#                 guess = input("Please enter your next guess").lower()
            
#             guess_entropies.append(get_word_entropy([guess], word_list, normalized = True, ascending = False)[0][1])

#         else:

#             if bias == "entropy":
                
#                 best_next_guesses = list(potential_next_guesses)                
#                 word_ratings = get_word_entropy(best_next_guesses, official_words, normalized = True, ascending = False) # "internal" ratings

#                 # Get max rated word
#                 max_rating = -np.inf
#                 for word, rating in word_ratings:
#                     if rating > max_rating:
#                         max_rating = rating

#                 # for word, rating in word_ratings:
#                 #     if rating == max_rating:
#                 #         guess = word
#                 if guess_num == max_guesses - 1:
#                     guess = input("Please enter your next guess").lower()
#                 else:
#                     guess = input("Please enter your next guess").lower()
                
#                 guess_entropies.append(get_word_entropy([guess], word_list, normalized = True, ascending = False)[0][1])

#                 if return_stats == False:
#                     if verbose == True:
#                         if len(word_ratings) <= 40:
#                             print(f"Potential next guesses:\n\t{word_ratings}\n")
#                             print (f"Words guessed so far:\n\t{guessed_words}.\n")
#                             record_list.append(f"Potential next guesses:\n\t{word_ratings}\n")
#                             record_list.append(f"Words guessed so far:\n\t{guessed_words}.\n")

#             if bias == "no_bias":
#                 best_next_guesses = set()
#                 for word in potential_next_guesses:
#                     for letter, freq in word_list_sorted_counts:
#                         if letter not in dont_guess_again:
#                             if len(next_letters) > 0:
#                                 if letter in next_letters:
#                                     if letter in word:
#                                         best_next_guesses.add(word)
#                                         break
#                             else:
#                                 if letter in word:
#                                     best_next_guesses.add(word)
#                                     break
                                
#                 if return_stats == False:
#                     if verbose == True:
#                         if len(best_next_guesses) <= 40:
#                             print(f"Potential next guesses:\n\t{best_next_guesses}\n")
#                             print (f"Words guessed so far:\n\t{guessed_words}.\n") 
#                             record_list.append(f"Potential next guesses:\n\t{best_next_guesses}\n")
#                             record_list.append(f"Words guessed so far:\n\t{guessed_words}.\n") 

#                 # guess = list(best_next_guesses)[0]
#                 if guess_num == max_guesses - 1:
#                     guess = input("Please enter your last guess").lower()
#                 else:
#                     guess = input("Please enter your next guess").lower()
                
#                 guess_entropies.append(get_word_entropy([guess], word_list, normalized = True, ascending = False)[0][1])

#         # print (guessed_words)

#         #### Guess has now been made -- what to do next
#         if guess_num == max_guesses: # if at max guesses allowed
#             stats_dict['target_guessed'] = False
#             if return_stats == False:
#                 if verbose == True:
#                     # print("-----------------------------\n")
#                     print(f"Unfortunately, the Wordle could not be solved in {max_guesses} guesses.\n")
#                     print(f"The target word was '{target}'. Better luck next time!\n")
#                     print("-----------------------------\n")
#                     record_list.append(f"Unfortunately, the Wordle could not be solved in {max_guesses} guesses.\n")
#                     record_list.append(f"The target word was '{target}'. Better luck next time!\n")
#                     record_list.append("-----------------------------\n")
#                 else:
#                     print(f"\nUnfortunately, the Wordle could not be solved in {max_guesses} guesses.")
#                     print(f"The target word was '{target}'. Better luck next time!\n")
#                     record_list.append(f"\nUnfortunately, the Wordle could not be solved in {max_guesses} guesses.")
#                     record_list.append(f"The target word was '{target}'. Better luck next time!\n")
#             break
#         else: # if not at max guesses yet allowed
#             # stats_dict['target_guessed'] = False
#             if return_stats == False:
#                 if verbose == True:
#                     print(f"Next guess, you entered:\n\t'{guess}'")
#                     print("\n-----------------------------\n")
#                     record_list.append(f"Next guess, you entered:\n\t'{guess}'")
#                     record_list.append("\n-----------------------------\n")

#         if guess == target:
#             guess_num += 1
#             stats_dict['target_guessed'] = True

#             if return_stats == False:
#                 print(f"Guess {guess_num}: '{guess}'\n")
#                 print(f"Congratulations! The Wordle has been solved in {guess_num} guesses!")
#                 record_list.append(f"Guess {guess_num}: '{guess}'\n")
#                 record_list.append(f"Congratulations! The Wordle has been solved in {guess_num} guesses!")

#                 if max_guesses - guess_num == 0:
#                     print(f"Lucky! It was the last guess.")
#                     record_list.append(f"Lucky! It was the last guess.")
#                 else:
#                     print(f"There were still {max_guesses - guess_num} guesses remaining.")
#                     record_list.append(f"There were still {max_guesses - guess_num} guesses remaining.")

#             if return_stats == False:   
#                 # stats_dict['target_guessed'] = True                 
#                 print(f"\nThe target word was '{target}'.")
#                 record_list.append(f"\nThe target word was '{target}'.")

#                 if first_guess[0] != "user_entered":
#                     print("\n-----------------------------")
#                     record_list.append("\n-----------------------------")
#                 else:
#                     print("\n-----------------------------")
#                     print("----------------------------------------------------------")
#                     record_list.append("\n-----------------------------")
#                     record_list.append("----------------------------------------------------------")
#             break

#     #### STATS STUFF    
#     mid_guesses_vows = 0
#     mid_guesses_cons = 0
#     avg_perf_letters = 0
#     avg_wrong_pos_letters = 0
#     avg_wrong_letters = 0

#     for i, word in enumerate(guessed_words):
#         mid_guesses_vows += count_vows_cons(word, y_vow = True)['vows']
#         mid_guesses_cons += count_vows_cons(word, y_vow = True)['cons']
        
#     # for i in range(0, len(guessed_words)):  
#         avg_perf_letters += perfect_letts_per_guess[i]
#         avg_wrong_pos_letters += wrong_pos_per_guess[i]
#         avg_wrong_letters += wrong_letts_per_guess[i]

#     stats_dict['mid_guesses_avg_vows'] = float(round(mid_guesses_vows / len(guessed_words), 2))
#     stats_dict['mid_guesses_avg_cons'] = float(round(mid_guesses_cons / len(guessed_words), 2))

#     stats_dict['avg_perf_letters'] = float(round(avg_perf_letters / len(guessed_words), 2))
#     stats_dict['avg_wrong_pos_letters'] = float(round(avg_wrong_pos_letters / len(guessed_words), 2))
#     stats_dict['avg_wrong_letters'] = float(round(avg_wrong_letters / len(guessed_words), 2))

#     # average number of words remaining after each guess -- the higher this is, the luckier the person got (the lower, the more guesses it took)
#     stats_dict['avg_remaining'] = float(round(np.mean(reduction_per_guess), 2))

#     # print(guess_entropies)

#     # avg entropy of each guessed word relative to all other words possible at that moment -- this should consistently be 100 for the algorithm, but will be different for user
#     if len(guess_entropies) > 1: # in case of guessing it correctly on the first try
#         avg_entropy = 0
#         for entropy in guess_entropies:
#             avg_entropy += entropy
#         stats_dict['avg_intermediate_guess_entropy'] = float(round(avg_entropy / len(guess_entropies), 2))
#     else:
#         stats_dict['avg_intermediate_guess_entropy'] = float(100)

#     stats_dict['num_guesses'] = float(guess_num)
#     stats_dict['bias'] = bias

#     if guess_num <= 6:
#         stats_dict['valid_success'] = True
#     else:
#         stats_dict['valid_success'] = False

#     stats_master = {}
#     if first_guess[0] == "user_entered":
#         if return_stats == False:
#             print(f"\nHere's the most statistically optimal way to play this:")
#             print("\n----------------------------------------------------------")
#             record_list.append(f"\nHere's the most statistically optimal way to play this:")
#             record_list.append("\n----------------------------------------------------------")

#         auto_stats_dict = wordle_wizard(word_list = word_list, max_guesses = max_guesses, 
#                           guess = guessed_words[0], target = target, bias = bias, 
#                           random_guess = random_guess, random_target = random_target,
#                           verbose = verbose, drama = drama, return_stats = return_stats, record = True)
        
#         if return_stats == True:
#             stats_dict['player'] = "user"
#             auto_stats_dict['player'] = "auto"
#             stats_dict['optimization'] = float(0) ### CHANGE THIS ACCORDING TO FORMULA THAT WORKS BEST FOR THIS -- RELATIVE TO BOT SCORE

#             for metric, result in stats_dict.items():
#                 stats_master[metric] = []
#                 stats_master[metric].append(result)
            
#             for metric, result in auto_stats_dict.items():
#                 stats_master[metric].append(result)

#     if record == True:
#         with open(f"{guessed_words[0]}_{target}_user.txt", "w") as fout:
#             for line in record_list:
#                 fout.write(line + "\n") # write each line of list of printed text to .txt file

#     if return_stats == True:
#         if first_guess[0] == "user_entered":
#             return stats_master
#         else:
#             return stats_dict

In [8]:
# user_wordle(word_list = official_words, max_guesses = 6, 
#         guess = "user_entered", target = "fifty", bias = 'entropy', 
#         random_guess = False, random_target = False, 
#         verbose = True, drama = 0, return_stats = False, record = True)

## Testing `wordle_wizard()`

In [12]:
wordle_wizard(word_list = official_words, max_guesses = 15, 
        guess = "later", target = "ready", bias = 'entropy', 
        random_guess = False, random_target = False, 
        verbose = True, drama = 0, return_stats = False, record = False)

-----------------------------

Guess 1: 'later'
Letters in correct positions:
	[]

Letters in incorrect positions:
	[('a', 1), ('e', 3), ('r', 4)]

Letters to guess again:
	['a', 'e', 'r']

Letters to not guess again:
	['l', 't']

At this point:
	2262, 97.96% of total words have been eliminated, and
	47, 2.04% of total words remain possible.

The top 40 potential next guesses are:
	[('arose', 100.0), ('arise', 93.23), ('snare', 85.32), ('adore', 77.61), ('scare', 77.33), ('opera', 75.26), ('share', 70.18), ('crane', 69.61), ('spare', 68.39), ('yearn', 65.38), ('afire', 57.48), ('reach', 54.47), ('recap', 52.59), ('ready', 50.8), ('cream', 48.64), ('repay', 48.45), ('grace', 48.17), ('heard', 47.7), ('argue', 47.41), ('drape', 45.91), ('brace', 45.72), ('dream', 41.96), ('grade', 41.49), ('creak', 39.98), ('grape', 39.23), ('beard', 39.04), ('bread', 39.04), ('erase', 38.57), ('rehab', 38.57), ('crave', 35.28), ('weary', 34.52), ('drake', 33.3), ('arena', 30.86), ('frame', 28.6), ('craz